In this programme we want to create segments of customers based on relevant buying behaviour. We are using a data set from here: http://archive.ics.uci.edu/ml/machine-learning-databases/00352/
More info on the data set can be found here: http://archive.ics.uci.edu/ml/datasets/Online+Retail

Each transaction has a customer ID and then associated information on that purchase such as price and quantity of products bought

In [130]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import itertools
from sklearn.mixture import GMM
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#Import the data file as a matrix
try:
    datafile = pd.read_csv('Retail_Data.csv')
except:
    print("The data cannot load")

#Create a placeholder for the no of segments we want to create
no_clusters = tf.placeholder(tf.float32, name = 'no_clusters')

#View basic info about the data
stats = datafile.describe()
stats



,CustomerID,total spend,No. of purchases,Average value of purchase
count,4372.000000,4372.000000,4372.000000,4372.000000
mean,15299.677722,322.008226,93.053294,7.978683
std,1722.390705,1284.783098,232.471608,144.963316
min,12346.000000,0.000000,1.000000,0.000000
25%,13812.750000,52.865000,17.000000,2.220000
50%,15300.500000,130.095000,42.000000,2.940000
75%,16778.250000,302.332500,102.000000,3.900000
max,18287.000000,41376.330000,7983.000000,8055.780000


In [27]:
#Select a number of data examples to view
samples = datafile.loc[[1,20,58, 287, 1025, 4000],:]
print(samples)

      CustomerID  total spend  No. of purchases  Average value of purchase
1          13047       798.30               196                       4.07
20         13408      1614.50               501                       3.22
58         15485       273.62                86                       3.18
287        17059      1371.39               516                       2.66
1025       15059       754.77               239                       3.16
4000       13607       308.76                98                       3.15


In [28]:
#Get the titles of the columns
column_names = datafile.columns
print(column_names)


Index(['CustomerID', 'total spend', 'No. of purchases',
       'Average value of purchase'],
      dtype='object')


In [30]:
#Apply log scale to the datat so it has a better distribution
log_data = np.log(datafile)
log_samples = np.log(samples)
print(log_samples)



      CustomerID  total spend  No. of purchases  Average value of purchase
1       9.476314     6.682484          5.278115                   1.403643
20      9.503607     7.386781          6.216606                   1.169381
58      9.647627     5.611740          4.454347                   1.156881
287     9.744433     7.223580          6.246107                   0.978326
1025    9.619731     6.626413          5.476464                   1.150572
4000    9.518340     5.732564          4.584967                   1.147402


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


Assign the value of the 25th percentile for the given feature to Q1. Use np.percentile for this.
Assign the value of the 75th percentile for the given feature to Q3. Again, use np.percentile.
Assign the calculation of an outlier step for the given feature to step.
Optionally remove data points from the dataset by adding indices to the outliers list.

In [132]:
#Create list to store outliers in
outliers_list = []
u_outliers = []
q1_list = {}
q3_list = {}
step_list = {}

#Find the 25th percentile figure for each column
for i in range(1,4):
    #find 25th pecentile
    q1 = np.percentile(log_data.loc[:,column_names[i]], 25)
    q1_list[column_names[i]] = [q1]
    #find 75th percentile
    q3 = np.percentile(log_data.loc[:,column_names[i]], 75)
    q3_list[column_names[i]] = [q3]
    #Calculate steps from IQR
    step = 1.5*(q3-q1)
    step_list[column_names[i]] = step
    #Find outliers
    outliers_rows = log_data.loc[~((log_data[column_names[i]] >= q1 - step) & (log_data[column_names[i]] <= q3 + step)), :]
    outliers_list.append(list(outliers_rows.index))

#Makes the outliers lists all one long list rather than them being separated by square brackets (d0nt understand!)
outliers = list(itertools.chain.from_iterable(outliers_list))  
u_outliers = list(set(outliers))

print("Q1 thresholds", q1_list)    
print("Q3 thresholds", q3_list)
print("List of steps", step_list)
print("list of unique outliers", u_outliers)

#remove all outliers from the dataset
good_data = log_data.drop(u_outliers)

Q1 thresholds {'total spend': [3.9677404060726249], 'No. of purchases': [2.8332133440562162], 'Average value of purchase': [0.79750719588418817]}
Q3 thresholds {'total spend': [5.7115265934774113], 'No. of purchases': [4.6249728132842707], 'Average value of purchase': [1.3609765531356006]}
List of steps {'total spend': 2.6156792811071794, 'No. of purchases': 2.6876392038420818, 'Average value of purchase': 0.84520403587711868}
list of unique outliers [514, 4, 3589, 518, 7, 8, 9, 1547, 524, 13, 2059, 4111, 3089, 1042, 19, 2067, 4114, 4118, 4122, 3099, 1051, 3101, 3102, 542, 3105, 2082, 1571, 3110, 2599, 1574, 1578, 2602, 557, 3117, 559, 48, 3629, 2615, 4156, 2622, 1090, 67, 3140, 69, 3652, 1608, 1096, 1611, 2126, 4178, 1106, 2643, 3154, 598, 604, 93, 3677, 1116, 1630, 3166, 4190, 3172, 4196, 4200, 108, 2157, 3692, 4210, 3193, 122, 2685, 1150, 1666, 132, 1669, 1668, 2181, 4231, 4232, 3211, 1165, 2702, 141, 1169, 2196, 4244, 1688, 2713, 1177, 2203, 2202, 3228, 1182, 671, 3743, 4258, 2729,

Now we will use k means clustering to find segments in the good_data and get a list of customer IDs for each of the segments. We will use the number of clusters with the greatest silhouette score

In [183]:
#Remove customer ID so it isnt used for the clustering
final_data = good_data.drop('CustomerID',1)

#Create clusters and apply predictions to data
kmeans = KMeans(n_clusters=2).fit(final_data)
preds = kmeans.predict(final_data)
kmeans.labels_
score = silhouette_score(final_data, preds, metric='euclidean')
centers = kmeans.cluster_centers_

true_centers = np.exp(centers)
print(true_centers)

#Get the indices for the 2 segments
segment_1_index = np.where(preds == 0)
segment_2_index = np.where(preds == 1)

segment_1_customer_list = good_data.iloc[segment_1_index]
segment_1_customers = np.exp(segment_1_customer_list)

segment_2_customer_list = good_data.iloc[segment_2_index]
segment_2_customers = np.exp(segment_2_customer_list)

segment_1_customers.describe()

[[ 347.83379466  117.24490576    2.96665568]
 [  50.216463     17.07382497    2.94127452]]


,CustomerID,total spend,No. of purchases,Average value of purchase
count,2033.000000,2033.000000,2033.000000,2033.000000
mean,15275.615839,456.911654,155.371864,3.143610
std,1715.566560,413.975357,146.509306,1.133583
min,12347.000000,89.850000,28.000000,0.980000
25%,13798.000000,204.020000,68.000000,2.370000
50%,15271.000000,305.960000,105.000000,2.930000
75%,16768.000000,543.380000,180.000000,3.690000
max,18287.000000,3852.330000,1284.000000,9.050000


In [185]:
segment_2_customers.describe()

,CustomerID,total spend,No. of purchases,Average value of purchase
count,2065.000000,2065.000000,2065.000000,2065.000000
mean,15323.720581,64.151720,21.308959,3.260305
std,1712.606720,39.778217,12.773308,1.520408
min,12348.000000,4.050000,2.000000,0.960000
25%,13859.000000,31.050000,11.000000,2.120000
50%,15325.000000,58.350000,20.000000,2.950000
75%,16778.000000,92.510000,30.000000,4.020000
max,18282.000000,203.450000,67.000000,8.990000
